# Utworzenie scrapera wydobywającego dane o opiniach na temat hotelów w wybranym mieście. Dane pochodzą z booking.com

In [1]:
from scrapy import Spider
from scrapy.crawler import CrawlerProcess

In [2]:
import logging
import os

!dir   --> dodanie wykrzyknika powoduje wykonanie polecenia powłoki

##### Funkcja próbująca usunąć plik jeśli już istnieje w celu uniknięcia błędów i dopisywania danych

In [3]:
def try_remove (filename):
    try:
        os.remove(filename)
    except OSError:
        pass

#### Właściwa klasa scrapująca:

In [38]:
class BookingComOPinions(Spider):
    name = 'BookingComOpinionsSpider' # nazwa jest obowiązkowa
    
    custom_settings = {
        'FEED_URI': '%(json_file)s', # zapisanie rezultatów do pliku %()s wrzua do pola argument który jest stringiem s, wykrywa jedyny string pobrany jako argument
        'FEED_FORMAT': 'json',       # wybór sposobu formatowania rezultatów
        'DOWNLOAD_DELAY': 3,         # trzeba uważać żeby nie dać za małego czasu bo można zostać zbanowanym
        'LOG_LEVEL': logging.DEBUG,  # poziom wartości informacji jakie będą wyświetlane (debug, warn error itp)
        'EXTENSIONS': {
            'scrapy.extensions.closespider.CloseSpider':1 # klasa która pozwala wykorzystać specjalny rodzaj wyjątku który powinien zatrzymać pająka
                                                          # CloseSpider przydaje się np. gdy dostaliśmy bana od strony więc nie ma sensu dalej szuka
        },
        'CLOSESPIDER_ITEMCOUNT': 10 # system zatrzyma się po porównaniu 10 hoteli
    }
    
    def parse (self, response):               # funkcja która będzie analizowała już wybraną stronę
        hotels = response.css('div.sr_item')  # lista bloków w których są opisane hotele
        for hotel in hotels:
            hotel_name = hotel.css('span.sr-hotel__name::text').get().strip() # hotel jest selektorem samym w sobie więc można na nim wykonać metodę .css() 
                                                                              # strip() obcinanie białych znaków z tekstu
            hotel_link = hotel.css('.hotel_name_link.url').pop()
            yield response.follow(hotel_link, self.parse_hotel,
                                 meta={'hotel': hotel_name})     # zakładamy historię meta-informacji która będzie działać w głąb
        
 #       for link in response.css('a[data-page-next]'): #pop() nie moze tu byc bo na ostatniej stronie brak elementu
 #           #zamiast tego petla ktora jest instrukcja warunkowa jesli element jest to wykonaj, a jak nie to nie wykonaj
 #           yield response.follow(link)
            
    def parse_hotel (self, response):
        hotel_name = response.meta['hotel']                         # wykorzystanie wcześniej utworzonych meta danych do wydobycia nazwy
        for reviews_link in response.css('a.show_all_reviews_btn'): # parsowanie ścieżki do opinii o hotelu
            yield response.follow(reviews_link, self.parse_reviews, # przekazanie nazwy funkcji bez wywołania
                                 meta={'hotel': hotel_name})        # forwardowanie metadanych (same się nie dziedziczą)
        
    def parse_reviews (self, response):
        hotel_name = response.meta['hotel']   
        items = response.css('li.review_item')
        for item in items:
            publish_date = item.css('meta[itemprop="datePublished"]'
                                   '::attr(content)').get('')        # funkcja get nie wydobędzie niczego (zwraca None) chyba że w nawiasach od get wstawi się wartość domyślnątu '',  gdy nie ma szukanego selektora
            reviewer = item.css('div.review_item_reviewer')
            rev_count = reviewer.css('div.'
                                    'review_item_user_review_count::text').get('')
            ...
            
            review = item.css('div.review_item_review')
            rating = item.css('meta[itemprop="ratingValue"]'
                                   '::attr(content)').get('')
            raw_tags = review.css('li.review_info_tag::text').getall()
            tags= list(filter(None, map(str.strip, raw_tags)))
            ...
                  # pierwszym parametrem filter może być None (wyłapuje puste stringi), a drugi to lista ?? lub tekst?? filtrowana/y
            yield{
                'hotel': hotel_name,
                'publish_date': publish_date,
                'rev_count': rev_count,
                'rating': rating,
                'tags': tags
            }
        
        for next_page in response.css('a#review_next_page_link'): # '#' bo szukamy po id  
            yield response.follow(next_page, self.parse_reviews,
                                 meta=response.meta)

In [5]:
ENTRY_URL = 'https://www.booking.com/searchresults.pl.html?city=-273837'

In [6]:
json_file = 'Oslo.json' # przypisanie nowej nazwy do pliku
try_remove(json_file)   # usuwanie pliku jeśli istnieje

##### Przygotowanie procesu przez crawlera, spider jest przygotowany i gotowy do pracy:

In [7]:
process = CrawlerProcess()
process.crawl(BookingComOPinions, start_urls = [ENTRY_URL],
             json_file = json_file)
# start_urls to lista tych adresów z których rozpoczynam przeszukiwanie

2019-04-11 14:22:07 [scrapy.utils.log] INFO: Scrapy 1.5.2 started (bot: scrapybot)
2019-04-11 14:22:07 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.9.0, Python 3.7.1 (default, Dec 10 2018, 22:54:23) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.1.1a  20 Nov 2018), cryptography 2.4.2, Platform Windows-10-10.0.17134-SP0
2019-04-11 14:22:07 [scrapy.crawler] INFO: Overridden settings: {'CLOSESPIDER_ITEMCOUNT': 10, 'DOWNLOAD_DELAY': 3, 'FEED_FORMAT': 'json', 'FEED_URI': '%(json_file)s', 'LOG_LEVEL': 10}
2019-04-11 14:22:07 [scrapy.extensions.telnet] INFO: Telnet Password: 7cbf95f83d42775b
2019-04-11 14:22:07 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.closespider.CloseSpider']
2019-04-11 14:22:07 [scrapy.middleware] 

#### Uruchomienie procesu scrapowania:

In [8]:
process.start()

2019-04-11 14:22:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.booking.com/searchresults.pl.html?city=-273837> (referer: None)
2019-04-11 14:22:12 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.booking.com/hotel/no/citybox-oslo.pl.html> from <GET https://www.booking.com/hotel/no/citybox-oslo.pl.html?from=searchresults%0A#hotelTmpl>
2019-04-11 14:22:15 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.booking.com/hotel/no/radisson-blu-oslo-alna.pl.html> from <GET https://www.booking.com/hotel/no/radisson-blu-oslo-alna.pl.html?from=searchresults%0A#hotelTmpl>
2019-04-11 14:22:19 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET https://www.booking.com/hotel/no/opera-apartments.pl.html> from <GET https://www.booking.com/hotel/no/opera-apartments.pl.html?from=searchresults%0A#hotelTmpl>
2019-04-11 14:22:23 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET h

2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/citybox-oslo.pl.html>
{'hotel': 'Citybox Oslo', 'publish_date': '2018-12-08', 'rev_count': '\n4 opinie\n', 'rating': '10', 'tags': ['Wyjazd wakacyjny', 'W parze', 'Mały pokój Dwuosobowy', '4 noclegi']}
2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/citybox-oslo.pl.html>
{'hotel': 'Citybox Oslo', 'publish_date': '2018-10-22', 'rev_count': '\n12 opinii\n', 'rating': '10', 'tags': ['Wyjazd wakacyjny', 'Rodzina z małymi dziećmi', 'Mały pokój Dwuosobowy', '1 nocleg']}
2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/citybox-oslo.pl.html>
{'hotel': 'Citybox Oslo', 'publish_date': '2018-10-05', 'rev_count': '\n26 opinii\n', 'rating': '9.6', 'tags': ['Wyjazd wakacyjny', 'W parze', 'Pokój Dwuosobowy', '2 noclegi']}
2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped

2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/citybox-oslo.pl.html>
{'hotel': 'Citybox Oslo', 'publish_date': '2018-09-10', 'rev_count': '\n8 opinii\n', 'rating': '10', 'tags': ['Wyjazd wakacyjny', 'Grupa', '2 pokoje', '2 noclegi']}
2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/citybox-oslo.pl.html>
{'hotel': 'Citybox Oslo', 'publish_date': '2018-09-06', 'rev_count': '\n15 opinii\n', 'rating': '7.5', 'tags': ['Wyjazd służbowy', 'Podróżujący ze znajomymi', 'Pokój z 2 łóżkami pojedynczymi', '2 noclegi', 'Wysłana przez urządzenie mobilne']}
2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/citybox-oslo.pl.html>
{'hotel': 'Citybox Oslo', 'publish_date': '2018-06-27', 'rev_count': '\n10 opinii\n', 'rating': '9.2', 'tags': ['Podróżujący w pojedynkę', 'Pokój jednoosobowy', '1 nocleg', 'Wysłana przez urządzenie mob

2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/citybox-oslo.pl.html>
{'hotel': 'Citybox Oslo', 'publish_date': '2015-07-05', 'rev_count': '\n14 opinii\n', 'rating': '6.7', 'tags': ['Wyjazd wakacyjny', 'W parze']}
2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/citybox-oslo.pl.html>
{'hotel': 'Citybox Oslo', 'publish_date': '2015-04-30', 'rev_count': '\n5 opinii\n', 'rating': '9.6', 'tags': ['Wyjazd wakacyjny', 'W parze']}
2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/citybox-oslo.pl.html>
{'hotel': 'Citybox Oslo', 'publish_date': '2015-04-20', 'rev_count': '\n9 opinii\n', 'rating': '7.9', 'tags': ['Wyjazd służbowy', 'Podróżujący w pojedynkę']}
2019-04-11 14:24:05 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/citybox-oslo.pl.html>
{'hotel': 'Citybox Oslo', 'publish_

2019-04-11 14:24:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/radisson-blu-oslo-alna.pl.html>
{'hotel': 'Radisson Blu Hotel Oslo Alna', 'publish_date': '2018-04-30', 'rev_count': '\n2 opinie\n', 'rating': '7.9', 'tags': ['Wyjazd wakacyjny', 'W parze', 'Pokój typu Standard', '2 noclegi', 'Wysłana przez urządzenie mobilne']}
2019-04-11 14:24:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/radisson-blu-oslo-alna.pl.html>
{'hotel': 'Radisson Blu Hotel Oslo Alna', 'publish_date': '2017-11-09', 'rev_count': '\n3 opinie\n', 'rating': '4.6', 'tags': ['Rodzina z małymi dziećmi', 'Pokój typu Standard', '1 nocleg']}
2019-04-11 14:24:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/radisson-blu-oslo-alna.pl.html>
{'hotel': 'Radisson Blu Hotel Oslo Alna', 'publish_date': '2019-03-12', 'rev_count': '\n2 opinie\n', 'rating': '7.5', 'tags': ['Wyjazd służbowy', 'Podróżujący

...

2019-04-11 14:24:43 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/anker-hotel.pl.html>
{'hotel': 'Anker Hotel', 'publish_date': '2018-08-19', 'rev_count': '\n2 opinie\n', 'rating': '8.3', 'tags': ['Wyjazd wakacyjny', 'Podróżujący ze znajomymi', 'Pokój z 2 łóżkami pojedynczymi', '3 noclegi', 'Wysłana przez urządzenie mobilne']}
2019-04-11 14:24:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/anker-hotel.pl.html>
{'hotel': 'Anker Hotel', 'publish_date': '2018-08-18', 'rev_count': '\n2 opinie\n', 'rating': '8.8', 'tags': ['Wyjazd wakacyjny', 'W parze', 'Pokój trzyosobowy', '7 noclegów']}
2019-04-11 14:24:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/anker-hotel.pl.html>
{'hotel': 'Anker Hotel', 'publish_date': '2018-08-02', 'rev_count': '\n13 opinii\n', 'rating': '8.8', 'tags': ['Wyjazd wakacyjny', 'Grupa', 'Pokój Dwuosobowy', '1 nocleg']}
2019-04-11 14:24:44 [

2019-04-11 14:24:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/anker-hotel.pl.html>
{'hotel': 'Anker Hotel', 'publish_date': '2019-02-26', 'rev_count': '\n2 opinie\n', 'rating': '9.6', 'tags': ['Wyjazd wakacyjny', 'W parze', 'Pokój dwuosobowy typu Budget (łóżko o szerokości 140 cm)', '2 noclegi']}
2019-04-11 14:24:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/anker-hotel.pl.html>
{'hotel': 'Anker Hotel', 'publish_date': '2019-02-18', 'rev_count': '\n2 opinie\n', 'rating': '10', 'tags': ['Wyjazd służbowy', 'Podróżujący w pojedynkę', 'Pokój jednoosobowy', '2 noclegi']}
2019-04-11 14:24:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/anker-hotel.pl.html>
{'hotel': 'Anker Hotel', 'publish_date': '2019-02-15', 'rev_count': '\n1 opinia\n', 'rating': '10', 'tags': ['Wyjazd służbowy', 'Podróżujący w pojedynkę', 'Pokój Dwuosobowy z 1 lub 2 łóżkami', '1 nocleg', 'W

2019-04-11 14:24:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/anker-hotel.pl.html>
{'hotel': 'Anker Hotel', 'publish_date': '2018-03-01', 'rev_count': '\n2 opinie\n', 'rating': '9.2', 'tags': ['Wyjazd służbowy', 'Podróżujący w pojedynkę', 'Pokój jednoosobowy', '1 nocleg', 'Wysłana przez urządzenie mobilne']}
2019-04-11 14:24:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/anker-hotel.pl.html>
{'hotel': 'Anker Hotel', 'publish_date': '2018-02-22', 'rev_count': '\n14 opinii\n', 'rating': '10', 'tags': ['Wyjazd wakacyjny', 'Podróżujący ze znajomymi', 'Pokój z 2 łóżkami pojedynczymi', '3 noclegi', 'Wysłana przez urządzenie mobilne']}
2019-04-11 14:24:44 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/anker-hotel.pl.html>
{'hotel': 'Anker Hotel', 'publish_date': '2018-02-06', 'rev_count': '\n2 opinie\n', 'rating': '6.5', 'tags': ['Wyjazd służbowy', 'Grupa', 'Pokój

2019-04-11 14:24:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/smarthotel-oslo.pl.html>
{'hotel': 'Smarthotel Oslo', 'publish_date': '2017-11-23', 'rev_count': '\n1 opinia\n', 'rating': '8.3', 'tags': ['Wyjazd służbowy', 'Podróżujący w pojedynkę', 'Mały pokój Dwuosobowy', '3 noclegi', 'Wysłana przez urządzenie mobilne']}
2019-04-11 14:24:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/smarthotel-oslo.pl.html>
{'hotel': 'Smarthotel Oslo', 'publish_date': '2017-11-06', 'rev_count': '\n40 opinii\n', 'rating': '9.6', 'tags': ['Wyjazd wakacyjny', 'Grupa', 'Mały pokój Dwuosobowy', '1 nocleg']}
2019-04-11 14:24:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/smarthotel-oslo.pl.html>
{'hotel': 'Smarthotel Oslo', 'publish_date': '2017-07-25', 'rev_count': '\n21 opinii\n', 'rating': '2.9', 'tags': ['W parze', 'Mały pokój Dwuosobowy', '2 noclegi']}
2019-04-11 14:24:47

2019-04-11 14:24:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/smarthotel-oslo.pl.html>
{'hotel': 'Smarthotel Oslo', 'publish_date': '2017-06-19', 'rev_count': '\n5 opinii\n', 'rating': '8.8', 'tags': ['Wyjazd wakacyjny', 'Rodzina z małymi dziećmi', 'Pokój rodzinny', '3 noclegi', 'Wysłana przez urządzenie mobilne']}
2019-04-11 14:24:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/smarthotel-oslo.pl.html>
{'hotel': 'Smarthotel Oslo', 'publish_date': '2017-06-02', 'rev_count': '\n1 opinia\n', 'rating': '6.7', 'tags': ['Wyjazd służbowy', 'Grupa', 'Mały pokój Dwuosobowy', '2 noclegi']}
2019-04-11 14:24:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.booking.com/reviews/no/hotel/smarthotel-oslo.pl.html>
{'hotel': 'Smarthotel Oslo', 'publish_date': '2016-05-26', 'rev_count': '\n5 opinii\n', 'rating': '7.5', 'tags': ['Wyjazd wakacyjny', 'W parze']}
2019-04-11 14:24:47 [scrapy.core.scraper] DE

##### get() , getall() redukuje selektory od stringu

##### .pop(0)  zdejmuje pierwszy element z listy

# Po wyscrapowaniu danych ze strony można przystąpić do przygotowania ich do analizy:

In [9]:
logging.getLogger().setLevel(logging.INFO) 
# zmiana levelu wyświetlania info

In [10]:
import pandas as pd

In [44]:
df = pd.read_json(json_file) 
# df bo dataframe

##### wyrzuamy napisy zawierające litery opiniae i spacje

In [45]:
df['rev_count'] = (df['rev_count'].str.strip()).str.strip(' opinea')
# df['rev_count'].str.strip() nie modyfikuje to dataframe, chyba że przypiszemy do tego samego obiektu                                                       

##### Konwersja do intów:

In [46]:
df['rev_count'] = df['rev_count'].astype(int) 

In [47]:
df['rev_count']

0       2
1       3
2       4
3      12
4      26
5      17
6       1
7      12
8       4
9      40
10     59
11     14
12     10
13     11
14     13
15     16
16      3
17     49
18      4
19     38
20     80
21     19
22     31
23     15
24      2
25      1
26     17
27      4
28      8
29     15
       ..
284     2
285     8
286     4
287     1
288     3
289    46
290     1
291     3
292    36
293     6
294     5
295     1
296     5
297     7
298     4
299     7
300    12
301     2
302     1
303     1
304     2
305    62
306    25
307    10
308     3
309    16
310     3
311    10
312     8
313     7
Name: rev_count, Length: 314, dtype: int32

##### Prygotowanie danych do sprawdzenia jakie tagi wpływają na rating:

In [42]:
subset = df[['rating', 'tags']] 
subdf = [subset]

In [ ]:
dum = subdf['tags'].map(lambda tags: '|'.join(tags)).str.get_dummies() 
# łączymy tagi dla danej opinii i dzielimy je separatorem  (w get_dummies() domyślny separator to |)

In [ ]:
indicators = pd.concat([subdf['rating'], dum], axis = 1) 
# axis mówi wzdłóż której osi ma nastąpić złączenie

In [ ]:
melted = pd.melt(indicators, id_vars=['rating'])
# id_vars zmienne które zostawiamy w spokoju

##### Selekcjonowanie według kryterium:

In [ ]:
positive = melted['value'] == 1
# wydobywamy wyrażenia z samymi 1 (a raczej odpowiedź na pytanie czy występuje w danym indeksie)
# taka lista może posłużyćza selektor danych w dataframe

In [ ]:
selected = melted[positive]

In [ ]:
aggregated = selected.groupby(['variable'])['rating'].agg(['mean','count'])
# groupby() wymaga podania listy tych kolumn wedłóg których ma nastąpić grupowanie
# fragment z rating to wycięcie interesującej kolumny
# agg() metoda agregacji przyjmująca jako parametr sposób agregowania, np. funkcja/słownik

##### na koniec będzie sortowanie w celu zwizualizowanie jaki tag wpływa najbardziej na rating:

In [ ]:
aggregated.sort_values(by = ['count']).tali(10) 
# sortowanie rosnąco według wartości count (10 ostatnich pozycji)

# KONIEC